# Import libraries and load dataset
We have 1.4M NYC taxi ride data. In the traning dataset, pickup location and time, drop-off location and time, Taxi vendor, and trip duration are given. From the testing datset, we have to estimate trip durarion when other data are given. First, we are going to explore the training dataset.

## Load dataset

In [88]:
# This NYC taxi data is avilable at kaggle
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Load training and testing data

In [89]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

## Explore training dataset

In [90]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


### Few observations
We have noticed few points here<br>
1) For all features there are no null values<br>
2) Vendor ID is given an integer<br>
3) Pickup and drop-off date and time for each trip are objects. We have to convert those in datetime<br>
4) Trip duration is given as an integer (Seconds)<br>
5) Store_and_fwd_flag (data send to server?): Yes or No<br>

Now, we are going to explore few columns to get a better idea

In [144]:
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime'] = pd.to_datetime(train['dropoff_datetime'])
train.groupby('vendor_id')['vendor_id'].count()*100/len(train)

vendor_id
1    46.504973
2    53.495027
Name: vendor_id, dtype: float64

In [92]:
train.groupby('passenger_count')['vendor_id'].count()*100/len(train)

passenger_count
0     0.004113
1    70.856220
2    14.418734
3     4.106280
4     1.947288
5     5.353465
6     3.313557
7     0.000206
8     0.000069
9     0.000069
Name: vendor_id, dtype: float64

In [93]:
train.groupby('store_and_fwd_flag')['vendor_id'].count()*100/len(train)

store_and_fwd_flag
N    99.44846
Y     0.55154
Name: vendor_id, dtype: float64

## Few more observation
1) There are two vendors, and both of them collect almost same number of data<br>
2) Mostly, there are 1 or 2 passenger in a ride. Passenger number varies 0-9.<br>
3) 99% taxi store data locally<br>

In [94]:
train['trip_duration'].describe()/60 # check trip time in minutes

count    24310.733333
mean        15.991538
std         87.290529
min          0.016667
25%          6.616667
50%         11.033333
75%         17.916667
max      58771.366667
Name: trip_duration, dtype: float64

In [108]:
train.loc[train.trip_duration>60*60, 'trip_duration'].nlargest(n=20, keep='first')/3600
# Check longest 20 trips, 23h means there may be some calculation error, this 


978383     979.522778
924150     618.781111
680594     569.327222
355003     538.815556
1234291     23.997778
295382      23.997500
73816       23.997222
59891       23.996389
1360439     23.995833
753765      23.994167
91717       23.993889
1221666     23.993889
66346       23.993611
1138915     23.993611
172662      23.991389
779030      23.991389
1269316     23.991389
1284067     23.991389
1213613     23.990833
1233528     23.990833
Name: trip_duration, dtype: float64

In [130]:
train.loc[(train.trip_duration>22*60*60) & (train.trip_duration<25*60*60), 'trip_duration'].count()

1902

### Trip duration values
Four trips have more than 500h trip duration<br>
1902 trip may suffer calculation error, around 24h trip duration

In [160]:
diff=(train['dropoff_datetime']-train['pickup_datetime']).astype('timedelta64[s]')-train['trip_duration']

In [161]:
diff.describe()

count    1458644.0
mean           0.0
std            0.0
min            0.0
25%            0.0
50%            0.0
75%            0.0
max            0.0
dtype: float64

Therefore, the time difference between pickup and dropoff is same as the trip duration. We do not have any other way to correct these values. We can delete those entries because we have lots of training data already.